In [20]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 

torch.set_warn_always(False)

In [21]:
print(torch.__version__)

2.3.1+cpu


In [22]:
with open(torch.__file__) as file : 
    print(file.read())


r"""
The torch package contains data structures for multi-dimensional
tensors and defines mathematical operations over these tensors.
Additionally, it provides many utilities for efficient serialization of
Tensors and arbitrary types, and other useful utilities.

It has a CUDA counterpart, that enables you to run your tensor computations
on an NVIDIA GPU with compute capability >= 3.0.
"""

import math
import os
import sys
import platform
import textwrap
import ctypes
import inspect
import threading

# multipy/deploy is setting this import before importing torch, this is the most
# reliable way we have to detect if we're running within deploy.
# https://github.com/pytorch/multipy/blob/d60f34ad38c371e441fe7ffdb77a3c3dda5a5d19/multipy/runtime/interpreter/interpreter_impl.cpp#L134-L137
def _running_with_deploy():
    return sys.modules.get("torch._meta_registrations", None) is object

from ._utils import _import_dotted_name, classproperty
from ._utils import _functionalize_sync as _sync


tensor is an multidimensional array, in practice we use 5 dim tensor. but for complex problems like satelite image processing we can use 200+ dim tensor 

* `torch` is the core module 
* `torch.autograd` compute gradient 
* `torch.nn` build any neural network 
* `torch.utils` for utility 

numpy doesn't support GPU that's why we use DL frameworks



In [23]:
# basic neural network 
class Model(nn.Module) : 
    def __init__(self, num_hidden) : 
        super(Model, self).__init__()
        self.layer1 = nn.Linear(28*28, 100)
        self.layer2 = nn.Linear(100, 50)
        self.layer3 = nn.Linear(50, 20)
        self.layer4 = nn.Linear(20, 1)
        self.num_hidden = num_hidden

    def forward(self, img) : 
        flattened = img.view(-1, 28*28)
        activation1 = F.relu(self.layer1(flattened))
        activation2 = F.relu(self.layer2(activation1))
        activation3 = F.relu(self.layer3(activation2))
        output = self.layer4(activation3)

        return output

### Tensor

we have to convert into a tensor for using pytorch

attributes 
- storage 
- stride 
- shape 
- device 
- size 
- grad 
- grad_fn 
- ndim

In [24]:
print(torch.tensor)

<built-in method tensor of type object at 0x00007FFCE2A889E0>


In [25]:
x = torch.tensor(9)
print(x, x.shape, x.ndim)

tensor(9) torch.Size([]) 0


In [26]:
x.item()

9

In [27]:
x.storage()

 9
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 1]

##### Create Tensor

In [28]:
x = torch.tensor([9])
print(x, x.shape, x.ndim)
print(x[0])
print(x.storage())
print(x.stride())

tensor([9]) torch.Size([1]) 1
tensor(9)
 9
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 1]
(1,)


In [29]:
x = torch.tensor([[9]])
print(x, x.shape, x.ndim)
print(x[0]) # loose 1 dim
print(x.storage())
print(x.stride())

tensor([[9]]) torch.Size([1, 1]) 2
tensor([9])
 9
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 1]
(1, 1)


In [30]:
# 1 dim 
x = torch.randint(0, 3, size = (2, ), generator = torch.manual_seed(42))
print(x, x.shape, x.ndim)

tensor([0, 2]) torch.Size([2]) 1


In [31]:
# 2 dim 
x = torch.randint(0, 3, size = (2, 2), generator = torch.manual_seed(42))
print("#"*10,'logical view', "#"*10)
print(x, x.shape, x.ndim)

########## logical view ##########
tensor([[0, 2],
        [1, 1]]) torch.Size([2, 2]) 2


In [32]:
# follows row major ordering 
x.storage()

 0
 2
 1
 1
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 4]

In [33]:
# stored in continuous memory location 
x = torch.Tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9]])
print(x[1])
print(x[1, 0])
print(x.stride())

tensor([0.4000, 0.5000, 0.6000])
tensor(0.4000)
(3, 1)


In [34]:
print(torch.sum(x, dim = 0))# col wise 
print(torch.sum(x, dim = 1)) # row wise 

tensor([1.2000, 1.5000, 1.8000])
tensor([0.6000, 1.5000, 2.4000])


In [35]:
# 2 x 2 x 3 x 2
# sum is a reduction operation
x = torch.Tensor([
    [
        [
            [0, 2], 
            [1, 1], 
            [0, 2]
        ], 
        [
            [1, 2], 
            [1, 2], 
            [1, 1]
        ]
    ],
    [
        [
            [2, 0], 
            [0, 1], 
            [2, 1]
        ], 
        [
            [0, 1], 
            [1, 2], 
            [1, 2]
        ]
    ]
])
print(torch.sum(x, dim=0))
print(torch.sum(x, dim=1))
print(torch.sum(x, dim=2))
print(torch.sum(x, dim=3))

tensor([[[2., 2.],
         [1., 2.],
         [2., 3.]],

        [[1., 3.],
         [2., 4.],
         [2., 3.]]])
tensor([[[1., 4.],
         [2., 3.],
         [1., 3.]],

        [[2., 1.],
         [1., 3.],
         [3., 3.]]])
tensor([[[1., 5.],
         [3., 5.]],

        [[4., 2.],
         [2., 5.]]])
tensor([[[2., 2., 2.],
         [3., 3., 2.]],

        [[2., 1., 3.],
         [1., 3., 3.]]])


In [36]:
x.storage()

 0.0
 2.0
 1.0
 1.0
 0.0
 2.0
 1.0
 2.0
 1.0
 2.0
 1.0
 1.0
 2.0
 0.0
 0.0
 1.0
 2.0
 1.0
 0.0
 1.0
 1.0
 2.0
 1.0
 2.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 24]

In [37]:
x.stride()

(12, 6, 2, 1)

In [38]:
x = torch.randint(0, 3, size = (2, 2, 2, 2), generator=torch.manual_seed(42))
print(x, x.shape, x.ndim)

tensor([[[[0, 2],
          [1, 1]],

         [[0, 2],
          [1, 2]]],


        [[[1, 2],
          [1, 1]],

         [[2, 0],
          [0, 1]]]]) torch.Size([2, 2, 2, 2]) 4
